# Coding a NN from Scratch

## 1. Import MNIST dataset from PyTorch

In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
from numba import jit, cuda

In [2]:
transform = torchvision.transforms.ToTensor()
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data/', train=False, download=True, transform=transform)

In [3]:
print(mnist_trainset,'\n\n', mnist_testset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor() 

 Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
# I just want the whole dataset in a np array, I do not want to use PyTorch for anything else than downloading MNIST

batch_size_train = 60000
batch_size_test = 10000

train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=batch_size_test, shuffle=True)

train_examples = enumerate(train_loader) #returns an iterable object that will spit the samples by batch
test_examples = enumerate(test_loader)

# Batch size is equal to the m examples in both sets (train and test), so we only need to iterate them one time
# Both batch indexes = 0 for the same reason stated before
train_batch_idx, (X_train_tensor, y_train_tensor) = next(train_examples)
test_batch_idx, (X_test_tensor, y_test_tensor) = next(test_examples)


In [5]:
print(f'X_train_tensor.shape: {X_train_tensor.shape}, y_train_tensor.shape: {y_train_tensor.shape}')
print(f'X_test_tensor.shape: {X_test_tensor.shape}, y_test.shape: {y_test_tensor.shape}')

X_train_tensor.shape: torch.Size([60000, 1, 28, 28]), y_train_tensor.shape: torch.Size([60000])
X_test_tensor.shape: torch.Size([10000, 1, 28, 28]), y_test.shape: torch.Size([10000])


In [6]:
import numpy as np

# Let's convert those tensors to np arrays only with rows and columns

def tensor_to_vector(tensor):
    vector_nd = np.array(tensor)
    vector = vector_nd.reshape(len(tensor),-1)
    return vector

X_train = tensor_to_vector(X_train_tensor)
X_test = tensor_to_vector(X_test_tensor)
y_train = np.array(y_train_tensor)
y_test = np.array(y_test_tensor)

In [7]:
print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')
print(f'X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}')

X_train.shape: (60000, 784), y_train.shape: (60000,)
X_test.shape: (10000, 784), y_test.shape: (10000,)


In [8]:
# We want examples by columns so let's transpose the Xs
X_train = X_train.T
X_test = X_test.T

print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')
print(f'X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}')

X_train.shape: (784, 60000), y_train.shape: (60000,)
X_test.shape: (784, 10000), y_test.shape: (10000,)


In [9]:
# The NN is going to have: Input Layer (784) - HL1(10) - HL2(10)
# Let's initalize all our parameters
def init_parameters():
    W1 = np.random.randn(10, 784) 
    b1 = np.random.randn(10, 1)
    W2 = np.random.randn(10, 10)
    b2 = np.random.randn(10, 1)
    return W1, b1, W2, b2
    
def ReLU(Z):
    a = np.maximum(0,Z) #this will return Z if the value is positive, otherwise 0
    return a

def SoftMax(Z):
    summation_of_exps = sum(np.exp(Z))
    return np.exp(Z)/summation_of_exps


def fw_prop(A0, W1, b1, W2, b2):
    #A0 = X
    Z1 = W1.dot(A0) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = SoftMax(Z2)
    return Z1, A1, Z2, A2

def one_hot_encoding(y):
    n_categories = y.max() + 1     # 0-9, 10 categories
    one_hot_y = np.zeros((y.size, n_categories))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T # We want the examples to be arranged by columns
    return one_hot_y

def d_ReLU(Z):
    return Z > 0


def b_prop(A0, Z1, A1, W2, Z2, A2, y):
    m = y.size
    one_hot_y = one_hot_encoding(y)
    
    dZ2 = (A2-one_hot_y)
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * d_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(A0.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2


def update_W(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2
    

In [10]:
def pred(A2):
    y_pred = np.argmax(A2,0)
    return y_pred

def accuracy(y_pred, y):
    accuracy = np.sum(y_pred == y) / y.size * 100
    return accuracy

def grad_desc(A0, y, iterations, alpha):
    # Initialize Parmeters
    W1, b1, W2, b2 = init_parameters()
    
    # Train Loop
    for i in range(iterations):
        # Forward Propagation
        Z1, A1, Z2, A2 = fw_prop(A0, W1, b1, W2, b2)
        # Back Propagation
        dW1, db1, dW2, db2 = b_prop(A0, Z1, A1, W2, Z2, A2, y)
        # Update Parameters
        W1, b1, W2, b2 = update_W(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i%20== 0:
            print(f'Iteration: {i}')
            y_pred = pred(A2)
            print(y_pred)
            score = accuracy(y_pred,y)
            print(f'Accuracy: {score}')
            
    return W1, b1, W2, b2
            
    

In [12]:
W1, b1, W2, b2 = grad_desc(X_train, y_train, 1000, 0.4)

Iteration: 0
[6 6 8 ... 0 8 8]
Accuracy: 11.808333333333334
Iteration: 20
[3 3 5 ... 1 3 0]
Accuracy: 19.18
Iteration: 40
[5 3 3 ... 1 5 0]
Accuracy: 21.923333333333332
Iteration: 60
[5 3 3 ... 1 5 2]
Accuracy: 19.878333333333334
Iteration: 80
[5 3 3 ... 1 5 2]
Accuracy: 21.498333333333335
Iteration: 100
[3 3 3 ... 1 3 2]
Accuracy: 22.89
Iteration: 120
[3 3 3 ... 1 3 2]
Accuracy: 23.898333333333333
Iteration: 140
[3 3 3 ... 1 3 2]
Accuracy: 25.001666666666665
Iteration: 160
[3 3 3 ... 1 3 2]
Accuracy: 26.205000000000002


KeyboardInterrupt: 